# Libraries

In [2]:
import pandas as pd
import requests

In [169]:
pd.set_option('display.max_columns', None) # display all columns in DF

# Connection to OpenFDA and data manipulation

In [276]:
payload = {'api_key': 'vJTYEEbNaMLaY4e8yZDGKYXwdmDdsPfZB05G87ea', 'limit': '1000'}
r = requests.get('https://api.fda.gov/drug/drugsfda.json?', params=payload)
data = r.json()

Flatting Nested JSON

In [274]:
df = pd.json_normalize(data['results'], max_level = 1)
df = df.drop(columns = ['submissions', 'products'])

Downloading as Excel-file

In [275]:
df.to_excel("output_openfda.xlsx")

In [175]:
df_test = df.explode('openfda.application_number') # can be used to remove list in data

# Connection to Medicaid and data manipulation

Creating a tables with all datasets and the related identifiers

In [380]:
r = requests.get('https://data.medicaid.gov/api/1/metastore/schemas/dataset/items')
data = r.json()

df = pd.json_normalize(data, max_level = 1)
df = df[df['title'].str.contains('State Drug Utilization Data')]

df.head(3)

,bureauCode,programCode,accessLevel,keyword,dataQuality,description,distribution,identifier,title,issued,modified,license,theme,landingPage,contactPoint.hasEmail,contactPoint.fn,publisher.name,@type,accrualPeriodicity,publisher.@type,temporal,references
114,[009:00],[009:076],public,"[pharmacy, medicaid reimbursements, drug utili...",NaN,Drug utilization data are reported by states f...,"[{'@type': 'dcat:Distribution', 'title': 'CSV'...",56adbcf4-f86b-5ab3-98fe-e713bbf99f12,State Drug Utilization Data 1996,2015-06-11T14:22:39+00:00,2021-02-18T03:23:39+00:00,https://www.usa.gov/government-works,[State Drug Utilization],NaN,no-reply@data.medicaid.gov,Medicaid.gov,data.medicaid.gov,dcat:Dataset,R/P10Y,org:Organization,NaN,NaN
115,[009:00],[009:076],public,"[pharmacy, medicaid reimbursements, drug utili...",NaN,Drug utilization data are reported by states f...,"[{'@type': 'dcat:Distribution', 'title': 'CSV'...",acbd1537-69b8-548d-a90f-eedb416acd71,State Drug Utilization Data 1992,2015-06-11T14:25:57+00:00,2021-02-18T02:59:11+00:00,https://www.usa.gov/government-works,[State Drug Utilization],NaN,no-reply@data.medicaid.gov,Medicaid.gov,data.medicaid.gov,dcat:Dataset,R/P10Y,org:Organization,NaN,NaN
116,[009:00],[009:076],public,"[pharmacy, medicaid reimbursements, drug utili...",NaN,Drug utilization data are reported by states f...,"[{'@type': 'dcat:Distribution', 'title': 'CSV'...",c39d3302-2d39-5209-9310-388ab1c8cbb8,State Drug Utilization Data 1999,2015-06-11T14:20:33+00:00,2021-02-18T03:45:30+00:00,https://www.usa.gov/government-works,[State Drug Utilization],NaN,no-reply@data.medicaid.gov,Medicaid.gov,data.medicaid.gov,dcat:Dataset,R/P10Y,org:Organization,NaN,NaN


Looping through datasets and the related splitting API requests

In [407]:
start_url = 'https://data.medicaid.gov/api/1/datastore/query/'
middle_url = 'ae4d5347-5137-5f6c-b66c3420fa0316d8'
end_url = '/0'

#r = requests.get(prefix_url + stem_url + suffix_url)
#data = r.json()

offset = 0

for i in range(249):
    print(i)
    try:
        if i == 0:
            r = requests.get(prefix_url + stem_url + suffix_url, timeout = 10)
            data = r.json()
            df = pd.json_normalize(data['results'], max_level = 1)
        else:
            offset = offset + 10000
            offset_url = '?offset=' + str(offset)
            r = requests.get(prefix_url + stem_url + suffix_url + offset_url, timeout = 10)
            data = r.json()
            temp_df = pd.json_normalize(data['results'], max_level = 1)
            df = df.append(temp_df, ignore_index = True)
    except:
        pass
        print('stop!')

In [408]:
print(df)

        utilization_type state          ndc labeler_code product_code package_size  year quarter suppression_used product_name units_reimbursed number_of_prescriptions total_amount_reimbursed medicaid_amount_reimbursed non_medicaid_amount_reimbursed
0                   FFSU    XX  49884018610        49884         0186           10  1996       4             true   METHYLDOPA                                                                                                                           
1                   FFSU    XX  00456032551        00456         0325           51  1996       3             true   LEVOTHROID                                                                                                                           
2                   FFSU    XX  00072220016        00072         2200           16  1996       1             true   EURAX LOTI                                                                                                                           


In [412]:
df.head(100)

,utilization_type,state,ndc,labeler_code,product_code,package_size,year,quarter,suppression_used,product_name,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed
0,FFSU,XX,49884018610,49884,0186,10,1996,4,true,METHYLDOPA,,,,,
1,FFSU,XX,00456032551,00456,0325,51,1996,3,true,LEVOTHROID,,,,,
2,FFSU,XX,00072220016,00072,2200,16,1996,1,true,EURAX LOTI,,,,,
3,FFSU,XX,00703536803,00703,5368,03,1996,3,true,SODIUM ACE,,,,,
4,FFSU,XX,00075366501,00075,3665,01,1996,1,true,TUSSAR SF,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,FFSU,XX,61392001554,61392,0015,54,1996,2,true,"VITA,MULTI",,,,,
96,FFSU,XX,00046042495,00046,0424,95,1996,4,true,INDERAL TA,,,,,
97,FFSU,XX,53303003040,53303,0030,40,1996,1,true,CARRINGTON,,,,,
98,FFSU,CT,51285097005,51285,0970,05,1996,2,true,DIGOXIN TA,,,,,


In [411]:
df.to_excel("output_mediciad.xlsx")

ValueError: This sheet is too large! Your sheet size is: 2490000, 15 Max sheet size is: 1048576, 16384